In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nmf.nmf_son.base import nmf_son
from nmf.nmf_son.andersen_acc import nmf_son_z_accelerated, nmf_son_all_accelerated
from nmf.nmf_son.her_acc import nmf_son_acc
from nmf.nmf_son.utils import *


np.random.seed(42)
np.set_printoptions(precision=3)

In [ ]:
def plot_scores(base_scores, scores_arr, ver_names):
    min_val = np.min(base_scores[~np.isnan(base_scores)])
    scores_arr = np.array(scores_arr)
    for x in scores_arr:
        min_x = np.min(x[~np.isnan(x)])
        if min_x < min_val:
            min_val = min_x


    fig, axs = plt.subplots(2, len(scores_arr), figsize=(20, 10), sharey='row')
    for idx in range(len(scores_arr)):
        axs[0, idx].set_yscale('log')
        axs[0, idx].set_title(ver_names[idx], fontsize=16)
        axs[0, idx].plot(base_scores, color='black', linewidth=3, label='Baseline')
        axs[0, idx].plot(scores_arr[idx], color='cyan', linewidth=1.5, label=ver_names[idx])
        axs[0, idx].legend()
        axs[0, idx].set_xlabel('Iterations')
        axs[0, idx].set_ylabel('$F(W, H)$')

        axs[1, idx].set_yscale('log')
        axs[1, idx].plot(base_scores - min_val, color='black', linewidth=3, label='Baseline')
        axs[1, idx].plot(scores_arr[idx] - min_val, color='cyan', linewidth=1.5, label=ver_names[idx])
        axs[1, idx].legend()
        axs[1, idx].set_xlabel('Iterations')
        axs[1, idx].set_ylabel('$F(W, H) - \min(F(W, H)$')


In [ ]:
X = np.load('../datasets/urban_small.npz')['X']
rank = 6

data = np.load(f'../saved_models/urban_small_r6_ini.npz')
ini_W = data['W']
ini_H = data['H']

reg = 2
save_filepath = '../saved_models/acc_test/urban_small_r{}_l{}_{}.npz'

In [ ]:
## Rank = 6

In [ ]:
aa_size = 2
max_iter = 1000

In [ ]:
%%time
Wb_base, Hb_base, Wl_base, Hl_base, fscores_base, gscores_base, lambda_vals_base = nmf_son(X, ini_W.copy(), ini_H.copy(), _lambda=reg, itermax=max_iter, verbose=True)

In [ ]:
%%time
Wb_acc, Hb_acc, Wl_acc, Hl_acc, fscores_acc, gscores_acc, lambda_vals_acc = nmf_son_acc(X, ini_W.copy(), ini_H.copy(), _lambda=reg, itermax=max_iter, verbose=True)

In [ ]:
%%time
Wb_z2, Hb_z2, Wl_z2, Hl_z2, fscores_z2, gscores_z2, lambda_vals_z2 = nmf_son_z_accelerated(X, ini_W.copy(), ini_H.copy(), _lambda=reg, itermax=max_iter, andersen_win=aa_size, verbose=True)

In [ ]:
%%time
Wb_all2, Hb_all2, Wl_all2, Hl_all2, fscores_all2, gscores_all2, lambda_vals_all2 = nmf_son_all_accelerated(X, ini_W.copy(), ini_H.copy(), _lambda=reg, itermax=max_iter, andersen_win=aa_size, verbose=True)

In [ ]:
# save_results(save_filepath.format(rank, reg, 'base'), Wb_base, Hb_base, Wl_base, Hl_base, fscores_base, gscores_base, lambda_vals_base)
# save_results(save_filepath.format(rank, reg, 'z2'), Wb_z2, Hb_z2, Wl_z2, Hl_z2, fscores_z2, gscores_z2, lambda_vals_z2)
# save_results(save_filepath.format(rank, reg, 'all2'), Wb_all2, Hb_all2, Wl_all2, Hl_all2, fscores_all2, gscores_all2, lambda_vals_all2)
# save_results(save_filepath.format(rank, reg, 'her'), Wb_acc, Hb_acc, Wl_acc, Hl_acc, fscores_acc, gscores_acc, lambda_vals_acc)
Wb_base, Hb_base, Wl_base, Hl_base, fscores_base, gscores_base, lambda_vals_base = load_results(save_filepath.format(rank, reg, 'base'))
Wb_z2, Hb_z2, Wl_z2, Hl_z2, fscores_z2, gscores_z2, lambda_vals_z2 = load_results(save_filepath.format(rank, reg, 'z2'))
Wb_all2, Hb_all2, Wl_all2, Hl_all2, fscores_all2, gscores_all2, lambda_vals_all2 = load_results(save_filepath.format(rank, reg, 'all2'))
Wb_acc, Hb_acc, Wl_acc, Hl_acc, fscores_acc, gscores_acc, lambda_vals_acc = load_results(save_filepath.format(rank, reg, 'her'))

In [ ]:
total_scores_base = fscores_base + lambda_vals_base * gscores_base
total_scores_z2 = fscores_z2 + lambda_vals_z2 * gscores_z2
total_scores_all2 = fscores_all2+ lambda_vals_all2 * gscores_all2
total_scores_acc = fscores_acc + lambda_vals_acc * gscores_acc

In [ ]:
plot_scores(total_scores_base, [total_scores_z2, total_scores_all2, total_scores_acc], ['Andersen Acceleration on $z$', 'Andersen Acceleration on $w_f, w_i, w_0, z$', 'HER'])

In [ ]:
plot_scores(fscores_base, [fscores_z2, fscores_all2, fscores_acc], ['andersen acc. z 2', 'andersen acc. all 2', 'her'])

In [ ]:
Wb_z3, Hb_z3, Wl_z3, Hl_z3, fscores_z3, gscores_z3, lambda_vals_z3 = nmf_son_z_accelerated(X, ini_W.copy(), ini_H.copy(), _lambda=reg, itermax=max_iter, andersen_win=3)
Wb_z4, Hb_z4, Wl_z4, Hl_z4, fscores_z4, gscores_z4, lambda_vals_z4 = nmf_son_z_accelerated(X, ini_W.copy(), ini_H.copy(), _lambda=reg, itermax=max_iter, andersen_win=4)

In [ ]:
save_results(save_filepath.format(rank, reg, 'z3'), Wb_z3, Hb_z3, Wl_z3, Hl_z3, fscores_z3, gscores_z3, lambda_vals_z3)
save_results(save_filepath.format(rank, reg, 'z4'), Wb_z4, Hb_z4, Wl_z4, Hl_z4, fscores_z4, gscores_z4, lambda_vals_z4)
# Wb_z3, Hb_z3, Wl_z3, Hl_z3, fscores_z3, gscores_z3, lambda_vals_z3 = load_results(acc_filepath.format(rank, iters, reg, 'z3'))
# Wb_z4, Hb_z4, Wl_z4, Hl_z4, fscores_z4, gscores_z4, lambda_vals_z4 = load_results(acc_filepath.format(rank, iters, reg, 'z4'))

In [ ]:
total_scores_z3 = fscores_z3 + lambda_vals_z3 * gscores_z3
total_scores_z4 = fscores_z4 + lambda_vals_z4 * gscores_z4

In [ ]:
plot_scores(total_scores_base, [total_scores_z2, total_scores_z3, total_scores_z4], ['andersen acc. z 2', 'andersen acc. z 3', 'andersen acc. z 4'])

In [ ]:
Wb_all3, Hb_all3, Wl_all3, Hl_all3, fscores_all3, gscores_all3, lambda_vals_all3 = nmf_son_all_accelerated(X, ini_W.copy(), ini_H.copy(), _lambda=reg, itermax=max_iter, andersen_win=3)
Wb_all4, Hb_all4, Wl_all4, Hl_all4, fscores_all4, gscores_all4, lambda_vals_all4 = nmf_son_all_accelerated(X, ini_W.copy(), ini_H.copy(), _lambda=reg, itermax=max_iter, andersen_win=4)

In [ ]:
save_results(save_filepath.format(rank, reg, 'all3'), Wb_all3, Hb_all3, Wl_all3, Hl_all3, fscores_all3, gscores_all3, lambda_vals_all3)
save_results(save_filepath.format(rank, reg, 'all4'), Wb_all4, Hb_all4, Wl_all4, Hl_all4, fscores_all4, gscores_all4, lambda_vals_all4)
# Wb_all3, Hb_all3, Wl_all3, Hl_all3, fscores_all3, gscores_all3, lambda_vals_all3 = load_results(acc_filepath.format(rank, iters, reg, 'all3'))
# Wb_all4, Hb_all4, Wl_all4, Hl_all4, fscores_all4, gscores_all4, lambda_vals_all4 = load_results(acc_filepath.format(rank, iters, reg, 'all4'))

In [ ]:
total_scores_all3 = fscores_all3 + lambda_vals_all3 * gscores_all3
total_scores_all4 = fscores_all4 + lambda_vals_all4 * gscores_all4

In [ ]:
plot_scores(total_scores_base, [total_scores_all2, total_scores_all3, total_scores_all4], ['andersen acc. all 2', 'andersen acc. all 3', 'andersen acc. all 4'])